In [1]:
import torch
import torch.nn as nn
import torch.cuda as cuda
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pdb
import pickle 
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from os import listdir
from os.path import isfile, join

import torch.nn.functional as F

import pandas as pd 
import scipy.io as sio
import csv

In [2]:
# cuda.check_error()

In [2]:
# set the dataDirectory and the path for saving data as .npy 
numSubjects = 17
dataDir
npyDataDir

### Generate the name of the files 

In [3]:
# generate the name of the files 
# the form is FARAH_sub0XY_LS(1/4)_tr(1:4)
# 16 subjects with LS1 and LS4, subject 17 has only LS1
# LS1:Loud speaker to the left 
# LS4:Loud speaker to the right 
fns = []
labels = []
for subject in range(1,numSubjects+1):
    for spk in [1,4]:
        for trial in range(1,4+1):
            dt = []
            fn = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial)
            fns.append(fn)

##### generate name of the subject based on the speaker side and the subject number 

In [4]:
# we give only the sujbect number and left or right speaker, and it generated the list of names
def gen_nameLists(subjectNumber, speakerSide):
    ssides = [1,4]
    if speakerSide not in ssides:
        print("speaker side is 1 or 4")
        #break
    
    namelists = []
    for trial in range(1,5):
        fname = 'FARAH_'+'sub0%0.2d'%subjectNumber+'_LS'+str(speakerSide)+'_'+str(trial)
        namelists.append(fname)
    return namelists

In [5]:
def read_by_name(fname):
    fpath = dataDir + fname
    dt = sio.loadmat(fpath)['y']
    dt = dt.transpose()
    return np.delete(dt, 29, axis=1)

## Saving each subject as a separate npy file 

In [6]:
# writing the individual files in a csv file
for fn in fns:
    dt = read_by_name(fn)
    path = npyDataDir +fn+'.npy'
    np.save(path, dt)
    # pd.DataFrame(dt).to_csv(path)

## Saving the dataset labels as a csv file

In [7]:
dict_labels = dict()
dict_electrode_labels = dict()
for subj in range(1,numSubjects+1):
    
    rspk = gen_nameLists(subj, speakerSide=1)
    lspk = gen_nameLists(subj, speakerSide=4)
    
    for name in rspk:
        dict_labels[name] = '+1'
        
    for name in lspk:
        dict_labels[name] = '-1'

with open(npyDataDir + 'dict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in dict_labels.items():
        writer.writerow([key, value])

#### Constructing the dataloader 

In [8]:
dict_labels = dict()
dict_electrode_labels = dict()
for subj in range(1,numSubjects+1):
    
    rspk = gen_nameLists(subj, speakerSide=1)
    lspk = gen_nameLists(subj, speakerSide=4)
    
    for name in rspk:
        dict_labels[name] = '+1'
        
    for name in lspk:
        dict_labels[name] = '-1'

with open(npyDataDir + 'dict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in dict_labels.items():
        writer.writerow([key, value])

In [9]:
class DirectionalMicrophoneDataset(Dataset):
    
    def __init__(self,subjects):
        
        # path: the path to the npy files 
        # subjects: the list of the subjects to be loaded
        
        self.path = npyDataDir
        self.subjects = subjects 
        self.samples = []
        
        #pdb.set_trace()
        for subject in self.subjects:
            for spk in [1, 4]:
                if spk == 1:
                    label = 0 # Right
                else:
                    label = 1 # Left
                
                for trial in [1, 2, 3, 4]:
                    fname = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial)
                    self.samples.append({
                        'fname': fname,
                        'label': label
                    })
                
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, indx):
        # 
        itempath = self.path + self.samples[indx]['fname'] +'.npy'
        signal = np.load(itempath) # loads the data 
        signal_label = self.samples[indx]['label'] # the corresponding label 
        
        return {'signal': signal, 
               'label': signal_label }
    

In [10]:
class RandomWindow(object):
    def __init__(self, window_size):
        self.window_size = window_size
        
    def __call__(self, sample):
        signal, label = sample['signal'], sample['label']
        
        # generate a random interval 
        random_index = np.random.randint(signal.shape[0]- self.window_size)
        dt = signal[random_index:random_index+self.window_size,:]
        return {'signal': dt, 'label': label}

In [11]:
class ToTensor(object):
    def __call__(self, sample):
        signal, label = sample['signal'], sample['label']
        dt = torch.from_numpy(signal.transpose())
        return {'signal': dt, 'label': label}

In [12]:
class TransformedDataset(Dataset):
    def __init__(self, dataset, *transforms):
        self.dataset = dataset
        self.transforms = transforms
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        for transform in self.transforms:
            sample = transform(sample)
        return sample

In [29]:
list_subjects = range(1,16)
dmd = DirectionalMicrophoneDataset(list_subjects)
window = RandomWindow(10000)
to_tensor = ToTensor()
dataset = TransformedDataset(dmd, window, to_tensor)

In [30]:
train_dl = DataLoader(dataset, batch_size=16, shuffle=True)

In [31]:
valid_subject = [17]
valid_dt= DirectionalMicrophoneDataset(valid_subject)
#to_tensor = ToTensor()
valid_dataset = TransformedDataset(valid_dt, window, to_tensor)
valid_dl = DataLoader(valid_dataset, batch_size=16, shuffle=True)

In [32]:
valid_dataset.dataset.samples

[{'fname': 'FARAH_sub017_LS1_1', 'label': 0},
 {'fname': 'FARAH_sub017_LS1_2', 'label': 0},
 {'fname': 'FARAH_sub017_LS1_3', 'label': 0},
 {'fname': 'FARAH_sub017_LS1_4', 'label': 0},
 {'fname': 'FARAH_sub017_LS4_1', 'label': 1},
 {'fname': 'FARAH_sub017_LS4_2', 'label': 1},
 {'fname': 'FARAH_sub017_LS4_3', 'label': 1},
 {'fname': 'FARAH_sub017_LS4_4', 'label': 1}]

In [33]:
len(valid_dataset.dataset)

8

## Constructing the network structure 

In [26]:
class DMNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Convolution Layer 1
        self.conv1 = nn.Conv1d(in_channels=127, out_channels=12, kernel_size=12)
        self.norm1 = nn.BatchNorm1d(12)
        self.maxpool1 = nn.MaxPool1d(2)
        self.relu1 = nn.ReLU()
        
        # Convolution Layer 2
        self.conv2 = nn.Conv1d(in_channels=12, out_channels=25, kernel_size=25)
        self.norm2 = nn.BatchNorm1d(25)
        self.maxpool2 = nn.MaxPool1d(2)
        self.relu2 = nn.ReLU()
        
        # Convolution Layer 3
        self.conv3 = nn.Conv1d(in_channels=25, out_channels=1, kernel_size=10)
        self.norm3 = nn.BatchNorm1d(1)
        self.maxpool3 = nn.MaxPool1d(4)
        self.relu3 = nn.ReLU()
        
        # Fully connected layers
        self.fc1 = nn.Linear(619, 100)
        self.fc2 = nn.Linear(100, 2)

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.maxpool1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.maxpool2(x)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.maxpool3(x)
        x = self.relu3(x)
        
        x = x.view(x.size(0), -1)
        
        # Fully connected layer 1
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training=True)
        
        # Fully connected layer 2
        x = self.fc2(x)
        
        return x

## Creating the object and defining the loss function

In [34]:
# The model
net = DMNetwork()

if cuda.is_available():
    print("cuda available")
    net = net.cuda()

# Our loss function
criterion = nn.CrossEntropyLoss()

# Our optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate) 

### training and validation steps 

In [ ]:
num_epochs = 2000

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []

for epoch in range(num_epochs):
    ############################
    # Train
    ############################    
    iter_loss = 0.0
    correct = 0
    iterations = 0
    
    net.train()                   # Put the network into training mode
    
    for samples in train_dl:
        # Convert torch tensor to Variable
        items = Variable(samples['signal'])
        classes = Variable(samples['label'])
        
        # If we have GPU, shift the data to GPU
        if cuda.is_available():
            items = items.cuda()
            classes = classes.cuda()
        
        optimizer.zero_grad()     # Clear off the gradients from any past operation
        outputs = net(items.float())      # Do the forward pass
        loss = criterion(outputs, classes) # Calculate the loss
        #pdb.set_trace()
        #iter_loss += loss.data[0] # Accumulate the loss
        iter_loss += loss.data.item()
        loss.backward()           # Calculate the gradients with help of back propagation
        optimizer.step()          # Ask the optimizer to adjust the parameters based on the gradients
        
        # Record the correct predictions for training data 
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        iterations += 1
    
    # Record the training loss
    train_loss.append(iter_loss/iterations)
    # Record the training accuracy
    #train_accuracy.append((100 * correct / len(mnist_train_loader.dataset)))
    print("Error: %.2f" % (iter_loss/iterations)+ "---- Accuracy: %.2f%%" % (100 * correct / len(train_dl.dataset)))
    #print("Accuracy: %.2f%%" % (100 * correct / len(train_dl.dataset)))
   

    # computing the validation loss 
    loss = 0.0
    correct = 0
    iterations = 0
    
    # putting the network into evaluation mode 
    net.eval()
    
    for validSample in valid_dl:
        
        items = Variable(validSample['signal'])
        classes = Variable(validSample['label'])
        
        # If we have GPU, shift the data to GPU
        if cuda.is_available():
            items = items.cuda()
            classes = classes.cuda()
        
        outputs = net(items.float())
        loss += criterion(outputs, classes).data.item()
        #loss += criterion(outputs, classes).data[0]
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations  +=1 
        
        
    valid_loss.append(loss/iterations)
    # Record the validation accuracy
    valid_accuracy.append(correct / len(valid_dl.dataset) * 100.0)
    
    print ('Val Loss: %.4f,---- Val Acc: %.4f' %(valid_loss[-1], valid_accuracy[-1]))
    pdb.set_trace()
        

In [1]:
import torch

In [2]:
torch.__version__

'1.0.0.dev20190326'

In [4]:
torch.cuda.is_available()

False